In [8]:
from time import time 
import pandas as pd
import warnings
from tqdm import tqdm
from gensim import corpora,similarities,models

In [2]:
train = pd.read_feather('../features/all_data/BASE_FIX_train.feather')
valid = pd.read_feather('../features/all_data/BASE_FIX_valid.feather')

In [3]:
all_data = pd.concat([train,valid])
del train,valid

In [6]:
all_data = all_data.reset_index(drop=True)

In [9]:
warnings.filterwarnings(action='ignore',category=UserWarning,module='gensim')
time1=time()

# test_data_all=pd.read_pickle("query_all.pickle")

all_data['tags'].fillna("-1",inplace=True)

class mycorpus(object):
    def __iter__(self):
        for index,doc in enumerate(tqdm(all_data['tags'])):
            yield doc.split(' ')
corp = mycorpus()
# dictionary.add_documents(corp)
dictionary=corpora.Dictionary(corp)
time2=time()
print(time2-time1)

100%|██████████| 99271300/99271300 [10:25<00:00, 158635.38it/s]

628.1332461833954


In [10]:
# test_data_all=pd.read_pickle("/home/kesci/work/query_all.pickle")

class MyCorpus(object):
    def __init__(self):
        print('MyCorpus')
    def __iter__(self):
        for index,doc in enumerate(tqdm(all_data['tags'])):
            yield dictionary.doc2bow(doc.split(' '))
corpus = MyCorpus()
# corpus = [dictionary.doc2bow(text) for text in self.text]
tfidf_model = models.TfidfModel(corpus, id2word=dictionary)
# corpus_tfidf = tfidf_model[corpus]

tfidf_model.save("../data/output/tfidf.model")
dictionary.save("../data/output/dictionary.model")

  0%|          | 30363/99271300 [00:00<05:26, 303625.22it/s]

MyCorpus


100%|██████████| 99271300/99271300 [05:06<00:00, 323794.38it/s]


In [13]:
time1=time()
warnings.filterwarnings(action='ignore',category=UserWarning,module='gensim')

# test_data_all=pd.read_pickle("/home/kesci/work/query_all.pickle")
corpus_tfidf = tfidf_model[corpus]

class MyCorpus(object):
    def __init__(self):
        a= 1
    def __iter__(self):
        for index,doc in enumerate(corpus_tfidf):
            yield doc
corpus3 = MyCorpus()
# corpus = [dictionary.doc2bow(text) for text in self.text]
lsi_model = models.LsiModel(corpus3, id2word=dictionary,chunksize=2500000,num_topics=50)
lsi_model.save("../data/output/lsi_all_query.lsi")
time2=time()
print((time2-time1)/60)
# corpus_tfidf = tfidf_model[corpus]

100%|██████████| 99271300/99271300 [59:46<00:00, 27677.66it/s]  


60.26381776730219


In [20]:
question = pd.read_csv('../data/input/questions.csv')

In [21]:
def get_arg_max(single_list):
    max_index=0
    max_num=single_list[0][1]
    for index in range(len(single_list)-1):
        if max_num<single_list[index+1][1]:
            max_num=single_list[index+1][1]
            max_index=index+1
    return max_index
# lsi_model=models.LsiModel.load("model/lsi_model/lsi_all_query_20.lsi")
# dictionary=corpora.Dictionary.load("model/dictionary.model")
# tfidf_model=corpora.Dictionary.load("model/tfidf.model")
all_data_lsi=[]
question['tags'] = question['tags'].fillna('-1')
for text in tqdm(question['tags']):
    single_row_text=dictionary.doc2bow(text.split(' '))
    single_row_lsi_list=list(lsi_model[tfidf_model[single_row_text]])
    if len(single_row_lsi_list)==0:
        all_data_lsi.append(0)
        continue
    single_row_lsi=get_arg_max(single_row_lsi_list)
    all_data_lsi.append(single_row_lsi)
question['tags_lsi']=all_data_lsi
question.to_csv("../data/output/question_lsi50.csv",index=None)

100%|██████████| 13523/13523 [00:02<00:00, 6325.80it/s]


,question_id,bundle_id,correct_answer,part,tags,tags_lsi
0,0,0,0,1,51 131 162 38,10
1,1,1,1,1,131 36 81,20
2,2,2,0,1,131 101 162 92,0
3,3,3,0,1,131 149 162 29,20
4,4,4,3,1,131 5 162 38,10
5,5,5,2,1,131 149 162 81,10
6,6,6,2,1,10 94 162 92,16
7,7,7,0,1,61 110 162 29,0
8,8,8,3,1,131 13 162 92,0
9,9,9,3,1,10 164 81,18


In [24]:
question['tags_lsi'].nunique()

48